# Parameters 

In [1]:
# Model
model_checkpoint = 'bert-base-uncased'
batch_size = 8
metric_name = "accuracy"

# Fold
fold = 0

# Experiment
labels = ["none", "attribution", "causal", "conditional", "contrast", "description", "equivalence", "fulfillment", "identity", "purpose", "summary", "temporal"]

# Import

In [2]:
import torch
import numpy as np
import random
import pandas as pd
from IPython.display import display, HTML

In [3]:
def import_fold(path, fold):
    train = pd.read_csv(f"{path}/train.{fold}.csv")
    test = pd.read_csv(f"{path}/train.{fold}.csv")
    return train, test

train_df, test_df = import_fold("data/export", fold)
display(train_df)

train_origin = train_df["origin"].tolist()
train_target = train_df["target"].tolist()
train_labels = train_df["label"].tolist()
test_origin = test_df["origin"].tolist()
test_target = test_df["target"].tolist()
test_labels = test_df["label"].tolist()

,origin,target,label
0,Canada confirms fifth case of Mad Cow Disease....,Swine flu: recent developments worldwide. A ma...,0
1,Swine flu: recent developments worldwide. A ma...,Canada confirms fifth case of Mad Cow Disease....,0
2,U.K. National Portrait Gallery threatens U.S. ...,Church of Scientology attempting to block 'int...,0
3,Church of Scientology attempting to block 'int...,U.K. National Portrait Gallery threatens U.S. ...,0
4,31 children rescued as UK police shut down glo...,U.K. National Portrait Gallery threatens U.S. ...,0
...,...,...,...
3163,Efforts to cap Deepwater Horizon oil spill del...,"Deepwater Horizon oil well finally dead, autho...",2
3164,"Volcano near Anchorage, Alaska erupts, airport...",Volcano erupts in southern Chile. This was not...,0
3165,Volcano erupts in southern Chile. This was not...,"Volcano near Anchorage, Alaska erupts, airport...",0
3166,More suspected plane debris washes ashore on M...,Malaysia says debris found in Maldives could b...,11


# Model 

## Metric

In [4]:
from sklearn.metrics import classification_report
import collections

#classification_threshold = 0.

def flatten(d, parent_key='', sep='__'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def compute_metrics(eval_pred):
    global labels
    predictions, true_labels = eval_pred
    # take most probable guess
    predictions = np.argmax(predictions, axis=-1)
    return flatten(classification_report(
        y_true=true_labels,
        y_pred=predictions,
        target_names=labels,
        zero_division=0,
        output_dict=True))

In [5]:
#TEST
#flatten(classification_report(
#    y_true=[0,1,2,3,4,5,6,7,8,9,10,11,12],
#    y_pred=[0,0,0,1,3,0,0,0,0,0,0,0,0],
#    target_names=labels,
#    zero_division=0,
#    output_dict=True))

## Model Settings

In [6]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

args = TrainingArguments(
    "semantic-test",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

## Tokenize

In [7]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)

In [8]:
train_encodings = tokenizer(train_origin, train_target, truncation=True, padding=True, return_token_type_ids=True)
test_encodings = tokenizer(test_origin, test_target, truncation=True, padding=True, return_token_type_ids=True)

## Print Examples

In [9]:
#train_encodings

In [10]:
def show_random_elements(origin_list, target_list, label_list, encodings, num_examples=10):
    global labels
    assert num_examples <= len(origin_list), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(origin_list)-1)
        while pick in picks:
            pick = random.randint(0, len(origin_list)-1)
        picks.append(pick)
    data = []
    for n in picks:
        data.append([n, origin_list[n], labels[label_list[n]], target_list[n], encodings.input_ids[n], encodings.token_type_ids[n], encodings.attention_mask[n]])
    df = pd.DataFrame(data, columns=['index', 'Origin', 'Label', 'Target', 'Input_ids', 'Token_type_ids', 'Attention_mask'])
    display(HTML(df.to_html()))

In [11]:
show_random_elements(train_origin, train_target, train_labels, train_encodings)
# Output adjustet to Folds
#show_random_elements(k_fold_origin[0][0], k_fold_target[0][0], k_fold_labels[0][0], train_encodings[0])

,index,Origin,Label,Target,Input_ids,Token_type_ids,Attention_mask
0,424,"Spanish Prime Minster condemns killing of Spanish councillor. Zapatero told the press that the ETA is defeated by democracy, and that the Government wants to roundly manifest that those who took part in the shooting will be arrested shortly, and to express its solidarity with the family and companions of Isaías, in both the PSE Basque Socialist Party and in the UGT trade union. March 7, 2008",causal,"Basque separatist group ETA declares ceasefire. The bombing was frowned upon by all involved in the peace process, including other Basque separatist parties in the region. September 6, 2010","[101, 3009, 3539, 8117, 6238, 28887, 2015, 4288, 1997, 3009, 10674, 1012, 23564, 17585, 3217, 2409, 1996, 2811, 2008, 1996, 27859, 2003, 3249, 2011, 7072, 1010, 1998, 2008, 1996, 2231, 4122, 2000, 2461, 2135, 19676, 2008, 2216, 2040, 2165, 2112, 1999, 1996, 5008, 2097, 2022, 4727, 3859, 1010, 1998, 2000, 4671, 2049, 14657, 2007, 1996, 2155, 1998, 11946, 1997, 18061, 7951, 1010, 1999, 2119, 1996, 8827, 2063, 13915, 6102, 2283, 1998, 1999, 1996, 1057, 13512, 3119, 2586, 1012, 2233, 1021, 1010, 2263, 102, 13915, 19802, 25879, 2923, 2177, 27859, 18806, 26277, 1012, 1996, 8647, 2001, 7335, 2588, 2011, 2035, 2920, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"
1,1582,"Scholastic sued for Harry Potter copyright infringement. The complaint stated that in both books, the protagonists are required to deduce the exact nature of the central task in the competition, and had done so in a bathroom. July 14, 2010",none,"Portuguese writer José Saramago dies at 87. As Saramago Foundation reported, today, June 18, José Saramago has died at 12:30 at his home in Lanzarote, at 87 years old, as the aftermath of a multiorgan failure, after a prolonged illness. June 18, 2010","[101, 24105, 12923, 2005, 4302, 10693, 9385, 20701, 1012, 1996, 12087, 3090, 2008, 1999, 2119, 2808, 1010, 1996, 21989, 2024, 3223, 2000, 2139, 8566, 3401, 1996, 6635, 3267, 1997, 1996, 2430, 4708, 1999, 1996, 2971, 1010, 1998, 2018, 2589, 2061, 1999, 1037, 5723, 1012, 2251, 2403, 1010, 2230, 102, 5077, 3213, 4560, 7354, 2863, 3995, 8289, 2012, 6584, 1012, 2004, 7354, 2863, 3995, 3192, 2988, 1010, 2651, 1010, 2238, 2324, 1010, 4560, 7354, 2863, 3995, 2038, 2351, 2012, 2260, 1024, 2382, 2012, 2010, 2188, 1999, 17595, 9057, 12184, 1010, 2012, 6584, 2086, 2214, 1010, 2004, 1996, 10530, 1997, 1037, 4800, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"
2,365,"Malaysia says debris found in Maldives could be from MH370. Debris has since washed ashore on at least three other Maldives locations, the Fehendhoo and Fulhahdhoo, but most of it isn't believed to be from an aircraft. August 10, 2015",causal,"Malaysia to investigate suspected plane debris washed ashore on several Maldives islands. Abdulla Rashe

## Create Dataset

In [12]:
class SemanticDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SemanticDataset(train_encodings, train_labels)
test_dataset = SemanticDataset(test_encodings, test_labels)

# Model

## Create Model

In [13]:
num_labels = len(labels)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Create Trainer

In [14]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

## Train

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,None Precision,None Recall,None F1-score,None Support,Attribution Precision,Attribution Recall,Attribution F1-score,Attribution Support,Causal Precision,Causal Recall,Causal F1-score,Causal Support,Conditional Precision,Conditional Recall,Conditional F1-score,Conditional Support,Contrast Precision,Contrast Recall,Contrast F1-score,Contrast Support,Description Precision,Description Recall,Description F1-score,Description Support,Equivalence Precision,Equivalence Recall,Equivalence F1-score,Equivalence Support,Fulfillment Precision,Fulfillment Recall,Fulfillment F1-score,Fulfillment Support,Identity Precision,Identity Recall,Identity F1-score,Identity Support,Purpose Precision,Purpose Recall,Purpose F1-score,Purpose Support,Summary Precision,Summary Recall,Summary F1-score,Summary Support,Temporal Precision,Temporal Recall,Temporal F1-score,Temporal Support,Accuracy,Macro avg Precision,Macro avg Recall,Macro avg F1-score,Macro avg Support,Weighted avg Precision,Weighted avg Recall,Weighted avg F1-score,Weighted avg Support
1,No log,0.882111,0.793292,0.982134,0.877670,2071,0.000000,0.000000,0.000000,15,0.212766,0.143369,0.171306,279,0.000000,0.000000,0.000000,35,0.000000,0.000000,0.000000,105,0.000000,0.000000,0.000000,30,0.500000,0.063725,0.113043,204,0.000000,0.000000,0.000000,24,0.000000,0.000000,0.000000,52,0.000000,0.000000,0.000000,15,0.000000,0.000000,0.000000,12,0.253846,0.303681,0.276536,326,0.690025,0.146659,0.124409,0.119880,3168,0.595651,0.690025,0.624577,3168
2,1.097100,0.672051,0.932781,0.978271,0.954985,2071,0.000000,0.000000,0.000000,15,0.290647,0.724014,0.414784,279,0.000000,0.000000,0.000000,35,0.000000,0.000000,0.000000,105,0.000000,0.000000,0.000000,30,1.000000,0.014706,0.028986,204,0.000000,0.000000,0.000000,24,0.000000,0.000000,0.000000,52,0.000000,0.000000,0.000000,15,0.000000,0.000000,0.000000,12,0.661074,0.604294,0.631410,326,0.766414,0.240375,0.193441,0.169180,3168,0.767800,0.766414,0.727668,3168
3,0.796900,0.584417,0.947051,0.984549,0.965436,2071,0.000000,0.000000,0.000000,15,0.488449,0.530466,0.508591,279,0.000000,0.000000,0.000000,35,0.000000,0.000000,0.000000,105,0.000000,0.000000,0.000000,30,0.425532,0.588235,0.493827,204,0.000000,0.000000,0.000000,24,0.000000,0.000000,0.000000,52,0.000000,0.000000,0.000000,15,0.000000,0.000000,0.000000,12,0.639535,0.843558,0.727513,326,0.815025,0.208381,0.245567,0.224614,3168,0.755340,0.815025,0.782583,3168
4,0.584100,0.469592,0.998033,0.980203,0.989038,2071,0.000000,0.000000,0.000000,15,0.740741,0.645161,0.689655,279,0.000000,0.000000,0.000000,35,0.000000,0.000000,0.000000,105,0.000000,0.000000,0.000000,30,0.377691,0.946078,0.539860,204,0.000000,0.000000,0.000000,24,0.000000,0.000000,0.000000,52,0.000000,0.000000,0.000000,15,0.000000,0.000000,0.000000,12,0.827586,0.957055,0.887624,326,0.857008,0.245338,0.294041,0.258848,3168,0.827158,0.857008,0.833399,3168
5,0.584100,0.412937,0.997569,0.990826,0.994186,2071,0.000000,0.000000,0.000000,15,0.763251,0.774194,0.768683,279,0.000000,0.000000,0.000000,35,0.000000,0.000000,0.000000,105,0.000000,0.000000,0.000000,30,0.403805,0.936275,0.564254,204,0.000000,0.000000,0.000000,24,0.000000,0.000000,0.000000,52,0.000000,0.000000,0.000000,15,0.000000,0.000000,0.000000,12,0.884181,0.960123,0.920588,326,0.875000,0.254067,0.305118,0.270643,3168,0.836342,0.875000,0.848687,3168


<ipython-input-4-c906eaaad80e>:10: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  if isinstance(v, collections.MutableMapping):


TrainOutput(global_step=1980, training_loss=0.7453565038815894, metrics={'train_runtime': 545.4221, 'train_samples_per_second': 3.63, 'total_flos': 0, 'epoch': 5.0})

## Evaluate

### Test Dataset

In [16]:
# Run Evaluation on Dataset with all trainers and return k-fold index of highest accuracy
def format_result(result):
    df = pd.DataFrame([result])
    display(HTML(df.T.to_html()))
    
format_result(trainer.evaluate(test_dataset))

,0
eval_loss,0.412937
eval_none__precision,0.997569
eval_none__recall,0.990826
eval_none__f1-score,0.994186
eval_none__support,2071.000000
eval_attribution__precision,0.000000
eval_attribution__recall,0.000000
eval_attribution__f1-score,0.000000
eval_attribution__support,15.000000
eval_causal__precision,0.763251


## Prediction

In [17]:
# Take best trainer
#trainer = trainers[best_fold_idx]

In [18]:
prediction_output = trainer.predict(test_dataset)

### Prediction metrics

In [19]:
df = pd.DataFrame([prediction_output.metrics])
display(df.T)

,0
test_loss,0.412937
test_none__precision,0.997569
test_none__recall,0.990826
test_none__f1-score,0.994186
test_none__support,2071.000000
test_attribution__precision,0.000000
test_attribution__recall,0.000000
test_attribution__f1-score,0.000000
test_attribution__support,15.000000
test_causal__precision,0.763251


### Prediction Examples

In [20]:
def example_prediction_output(result, origin_list, target_list, label_list, success=True, max_number=10):
    global labels
    # Real Relation
    data = []
    for n in range(len(origin_list)):
        data.append([origin_list[n], target_list[n], labels[label_list[n]]])
    df_real = pd.DataFrame(data, columns=['Origin', 'Target', 'Label'])
    # Predicted Label
    label_data = []
    for n in result.label_ids:
        i = np.argmax(result.predictions[n], axis=-1)
        label_data.append([labels[i]])
    df_pred_labels = pd.DataFrame(label_data, columns=["Prediction"])
    # Prediction Metrics
    df_predictions = pd.DataFrame(result.predictions, columns=labels)
    df_predictions = df_predictions.add_prefix("Pred_")
    df = pd.concat([df_real, df_pred_labels, df_predictions], axis=1)
    # Filter complete output to show example
    if success:
        # If possible only show examples with right predicted label that is not none
        df_tmp = df[(df.Label == df.Prediction) & (df.Label != "none")]
        if(df_tmp.size == 0):
            # if there is no match without label "none", print the "none"-matches
            df = df[(df.Label == df.Prediction)]
        else:
            df = df_tmp
    else:
        # If possible only show examples with right predicted label that is not none
        df_tmp = df[(df.Label != df.Prediction) & (df.Label != "none") & (df.Prediction != "none")]
        if(df_tmp.size == 0):
            df_tmp = df[(df.Label != df.Prediction) & ((df.Label != "none") | (df.Prediction != "none"))]
            if(df_tmp.size == 0):
                df = df[df.Label != df.Prediction]
            else:
                df = df_tmp
        else:
            df = df_tmp
    display(HTML(df.head(max_number).to_html()))

#### Successes

In [21]:
example_prediction_output(prediction_output, test_origin, test_target, test_labels)

,Origin,Target,Label,Prediction,Pred_none,Pred_attribution,Pred_causal,Pred_conditional,Pred_contrast,Pred_description,Pred_equivalence,Pred_fulfillment,Pred_identity,Pred_purpose,Pred_summary,Pred_temporal
0,"Canada confirms fifth case of Mad Cow Disease. This is Canada's fifth confirmed case of the disease since May 2003, and officials have tested more than 100,000 cows since the first case of the disease turned up in Alberta. April 14, 2006","Swine flu: recent developments worldwide. A majority of the cases were reported by the United States, which now has at least 10,053 confirmed infections from the disease. May 19, 2009",none,none,6.796221,-0.310706,-0.037133,-0.411199,-1.136399,-0.951643,-1.087620,-0.564183,-1.804138,-0.369897,-0.601369,0.821839
1,"Swine flu: recent developments worldwide. A majority of the cases were reported by the United States, which now has at least 10,053 confirmed infections from the disease. May 19, 2009","Canada confirms fifth case of Mad Cow Disease. This is Canada's fifth confirmed case of the disease since May 2003, and officials have tested more than 100,000 cows since the first case of the disease turned up in Alberta. April 14, 2006",none,none,6.937126,-0.426267,0.189443,-0.461942,-1.044270,-0.933352,-0.989166,-0.558361,-1.735762,-0.455372,-0.602375,0.421047
2,"U.K. National Portrait Gallery threatens U.S. citizen with legal action over Wikimedia images. David Gerard, former Press Officer for Wikimedia UK, the U.K. chapter of the Wikimedia Foundation, which has been involved with the Wikipedia Loves Art contest to create free content photographs of exhibits at the Victoria and Albert Museum, stated on Slashdot that The NPG actually acknowledges in their letter that the poster's actions were entirely legal in America, and that they're making a threat just because they think they can. July 11, 2009","Church of Scientology attempting to block 'intolerant' German film. Carl Bergengruen, the head of TV movies for Südwestrundfunk (SWR or Southwest Broadcasting), the public broadcaster backing the project defended their secrecy practices saying, Scientology kept trying to use a variety of methods to find out details about the project. March 15, 2010",none,none,7.684356,-0.934911,0.937986,-0.539007,-0.814859,-1.310764,-0.984200,-0.921400,-1.907234,-0.786805,-1.076709,-0.094654
3,"Church of Scientology attempting to block 'intolerant' German film. Carl Bergengruen, the head of TV movies for Südwestrundfunk (SWR or Southwest Broadcasting), the public broadcaster backing the project defended their secrecy practices saying, Scientology kept trying to use a variety of methods to find out details about the project. March 15, 2010","U.K. National Portrait Gallery threatens U.S. citizen with legal action over Wikimedia images. David Gerard, former Press Officer for Wikimedia UK, the U.K. chapter of the Wikimedia Foundation, which has been involved with the Wikipedia Loves Art contest to create free content photographs of exhibits at the Victoria and Albert Museum, stated on Slashdot that The NPG actually acknowledges in their letter that the poster's actions were entirely legal in America, and that they're making a threat just because they think they can. July 11, 2009",none,none,7.430922,-0.593533,0.461762,-0.423344,-0.868214,-1.037013,-0.982202,-0.813418,-1.888756,-0.603058,-0.726541,0.050769
4,"31 children rescued as UK police shut down global paedophile ring. Cox himself was found to have 75,960 indecent and explicit images on his computer and police found evidence he supplied at least 11,491 images to other paedophiles over the chatroom, on which he used the identity Son of God. He admitted to nine counts of possessing or distributing images of children. June 18, 2007","U.K. National Portrait Gallery threatens U.S. citizen with legal action over Wikimedia images. David Gerard, former Press Officer for Wikimedia UK, the U.K. chapter of the Wikimedia Foundation, which has been involved with the Wikipe

#### Fails

In [22]:
example_prediction_output(prediction_output, test_origin, test_target, test_labels, False)

,Origin,Target,Label,Prediction,Pred_none,Pred_attribution,Pred_causal,Pred_conditional,Pred_contrast,Pred_description,Pred_equivalence,Pred_fulfillment,Pred_identity,Pred_purpose,Pred_summary,Pred_temporal
25,"Fatah assaults European Union office. 'This article has been archived as published, however, the gunmen, according to sources listed, were members of the Al Aqsa Martyrs' Brigades.' In a reaction to the ongoing controversy over the September publication of cartoons by the Danish newspaper Jyllands-Posten, gunmen from the Fatah assaulted a European Union (EU) office in the Gaza Strip on Monday, prompting Denmark to warn citizens to avoid travel to Saudi Arabia. January 27, 2006","New Zealand newspapers publish ""Mohammad Cartoons"". Two New Zealand newspapers, Fairfax-owned The Dominion Post and The Christchurch Press, today controversially published all 12 cartoons of the Prophet Mohammad, which have triggered international outrage. February 4, 2006",causal,none,-0.875474,-1.500314,1.978083,-0.978482,0.898743,-1.094446,1.575956,-1.153589,-0.140207,-1.223209,-1.754261,-0.816238
26,"New Zealand newspapers publish ""Mohammad Cartoons"". Two New Zealand newspapers, Fairfax-owned The Dominion Post and The Christchurch Press, today controversially published all 12 cartoons of the Prophet Mohammad, which have triggered international outrage. February 4, 2006","Fatah assaults European Union office. 'This article has been archived as published, however, the gunmen, according to sources listed, were members of the Al Aqsa Martyrs' Brigades.' In a reaction to the ongoing controversy over the September publication of cartoons by the Danish newspaper Jyllands-Posten, gunmen from the Fatah assaulted a European Union (EU) office in the Gaza Strip on Monday, prompting Denmark to warn citizens to avoid travel to Saudi Arabia. January 27, 2006",causal,none,-0.681203,-1.553250,2.171267,-0.872714,0.871357,-1.024705,1.648160,-1.057060,-0.299367,-1.301426,-1.839841,-0.661958
37,"Protests erupt in Pakistan over US air strike. Many Pakistanis took to the streets Saturday to demonstrate against the U.S. air strike of a village on the border of Afghanistan. January 14, 2006","At least 40 killed by bombing in Pakistan. The attack took place on the outskirts of Dara Adamkhel, a town which is close to the border with Afghanistan. March 2, 2008",causal,none,-1.815199,-1.472281,1.408857,-0.599616,0.705016,-0.678495,1.460095,-0.932614,-0.040450,-1.209824,-1.493517,0.211566
38,"At least 40 killed by bombing in Pakistan. The attack took place on the outskirts of Dara Adamkhel, a town which is close to the border with Afghanistan. March 2, 2008","Protests erupt in Pakistan over US air strike. Many Pakistanis took to the streets Saturday to demonstrate against the U.S. air strike of a village on the border of Afghanistan. January 14, 2006",causal,none,-0.386256,-1.546684,2.277749,-0.919457,0.997434,-1.010176,1.454126,-1.063916,-0.321050,-1.160465,-1.814899,-0.998759
54,"British Prime Minister Tony Blair suffers defeat in vote on terror laws. Opponents believe that by effectively giving the government the right to imprison for up to three months anyone who it alleged to have been involved in terrorism, without having to present any charges in court to justify the detention, the bill could have led to abuses of power. November 09, 2005","Documents allege U.S. Army kidnapped wives of enemy fighters. All of the released women had been accused of aiding terrorists or planting explosives, but an Iraqi government commission found there was insufficient evidence to continue holding them. January 28,2006",causal,none,-0.464692,-1.560784,2.302031,-0.969854,0.977077,-1.154420,1.414792,-1.136056,-0.351048,-1.312699,-1.828428,-0.858122
61,"Bomb blast kills 5 policemen in Islamabad, Pakistan. At least five policemen are dead after a bombing took place near the Islamabad, Pakistan. April 3, 2009","Seven policemen killed in Afghanistan after bomb explosion. At least seve

### Prediction output

In [23]:
def format_prediction_output(result, origin_list, target_list, label_list):
    global labels
    # Real Relation
    data = []
    for n in range(len(origin_list)):
        data.append([origin_list[n], target_list[n], labels[label_list[n]]])
    df_real = pd.DataFrame(data, columns=['Origin', 'Target', 'Label'])
    # Predicted Label
    label_data = []
    for n in result.label_ids:
        i = np.argmax(result.predictions[n], axis=-1)
        label_data.append([labels[i]])
    df_pred_labels = pd.DataFrame(label_data, columns=["Prediction"])
    # Prediction Metrics
    df_predictions = pd.DataFrame(result.predictions, columns=labels)
    df_predictions = df_predictions.add_prefix("Pred_")
    df = pd.concat([df_real, df_pred_labels, df_predictions], axis=1)
    display(HTML(df.to_html()))
    
format_prediction_output(prediction_output, test_origin, test_target, test_labels)

,Origin,Target,Label,Prediction,Pred_none,Pred_attribution,Pred_causal,Pred_conditional,Pred_contrast,Pred_description,Pred_equivalence,Pred_fulfillment,Pred_identity,Pred_purpose,Pred_summary,Pred_temporal
0,"Canada confirms fifth case of Mad Cow Disease. This is Canada's fifth confirmed case of the disease since May 2003, and officials have tested more than 100,000 cows since the first case of the disease turned up in Alberta. April 14, 2006","Swine flu: recent developments worldwide. A majority of the cases were reported by the United States, which now has at least 10,053 confirmed infections from the disease. May 19, 2009",none,none,6.796221,-0.310706,-0.037133,-0.411199,-1.136399,-0.951643,-1.087620,-0.564183,-1.804138,-0.369897,-0.601369,0.821839
1,"Swine flu: recent developments worldwide. A majority of the cases were reported by the United States, which now has at least 10,053 confirmed infections from the disease. May 19, 2009","Canada confirms fifth case of Mad Cow Disease. This is Canada's fifth confirmed case of the disease since May 2003, and officials have tested more than 100,000 cows since the first case of the disease turned up in Alberta. April 14, 2006",none,none,6.937126,-0.426267,0.189443,-0.461942,-1.044270,-0.933352,-0.989166,-0.558361,-1.735762,-0.455372,-0.602375,0.421047
2,"U.K. National Portrait Gallery threatens U.S. citizen with legal action over Wikimedia images. David Gerard, former Press Officer for Wikimedia UK, the U.K. chapter of the Wikimedia Foundation, which has been involved with the Wikipedia Loves Art contest to create free content photographs of exhibits at the Victoria and Albert Museum, stated on Slashdot that The NPG actually acknowledges in their letter that the poster's actions were entirely legal in America, and that they're making a threat just because they think they can. July 11, 2009","Church of Scientology attempting to block 'intolerant' German film. Carl Bergengruen, the head of TV movies for Südwestrundfunk (SWR or Southwest Broadcasting), the public broadcaster backing the project defended their secrecy practices saying, Scientology kept trying to use a variety of methods to find out details about the project. March 15, 2010",none,none,7.684356,-0.934911,0.937986,-0.539007,-0.814859,-1.310764,-0.984200,-0.921400,-1.907234,-0.786805,-1.076709,-0.094654
3,"Church of Scientology attempting to block 'intolerant' German film. Carl Bergengruen, the head of TV movies for Südwestrundfunk (SWR or Southwest Broadcasting), the public broadcaster backing the project defended their secrecy practices saying, Scientology kept trying to use a variety of methods to find out details about the project. March 15, 2010","U.K. National Portrait Gallery threatens U.S. citizen with legal action over Wikimedia images. David Gerard, former Press Officer for Wikimedia UK, the U.K. chapter of the Wikimedia Foundation, which has been involved with the Wikipedia Loves Art contest to create free content photographs of exhibits at the Victoria and Albert Museum, stated on Slashdot that The NPG actually acknowledges in their letter that the poster's actions were entirely legal in America, and that they're making a threat just because they think they can. July 11, 2009",none,none,7.430922,-0.593533,0.461762,-0.423344,-0.868214,-1.037013,-0.982202,-0.813418,-1.888756,-0.603058,-0.726541,0.050769
4,"31 children rescued as UK police shut down global paedophile ring. Cox himself was found to have 75,960 indecent and explicit images on his computer and police found evidence he supplied at least 11,491 images to other paedophiles over the chatroom, on which he used the identity Son of God. He admitted to nine counts of possessing or distributing images of children. June 18, 2007","U.K. National Portrait Gallery threatens U.S. citizen with legal action over Wikimedia images. David Gerard, former Press Officer for Wikimedia UK, the U.K. chapter of the Wikimedia Foundation, which has been involved with the Wikipe